# Pipeline A

In [ ]:
##> Automated module reload
%load_ext autoreload
%autoreload 2 #

In [38]:
import os
import sys
sys.path.append("../")

import datetime
from datetime import datetime, timedelta, date

from pandas import read_csv, to_datetime
from geopandas import GeoDataFrame, read_file
import pandas as pd
import numpy as np
import movingpandas as mpd
from shapely.geometry import Point

from dtaidistance import dtw_ndim
from dtaidistance import dtw

from sklearn.cluster import HDBSCAN, KMeans
import matplotlib.pyplot as plt

import hvplot
import hvplot.pandas
import holoviews as hv
from holoviews import opts, dim;
import matplotlib.pyplot as plt
from holoviews.element.tiles import EsriImagery

opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=500, frame_height=400))
hvplot_defaults = {'tiles':'ESRI', 'cmap':'Viridis', 'colorbar':True}

In [39]:
from src.macros.macros1 import *

In [40]:
from src.preprocess.load import MMSIperDayFeatures, TYPES_TO_INT, get_data_for_time_frame, query_by_type

## 1. LOAD the data

In [41]:
START_DATE = date(year=2022, month=7, day=5)
END_DATE = date(year=2022, month=7, day=5)

In [42]:
src = r"../assets/csv/"
data = get_data_for_time_frame(src, START_DATE, END_DATE)

In [43]:
len(data)

57

data: list
+ mmsi: int
+ day: datetime.date
+ static: DataFrame
+ own: DataFrame
+ s2s: DataFrame

## 2. select by TYPE

In [44]:
selected_type = "SAILING_FAMILY"
data_selected = query_by_type(data, selected_type)
print(f"Number of {selected_type} ships found: {len(data_selected)} ships")

Number of SAILING_FAMILY ships found: 17 ships


## 3. prepare DATA

- Update the trajectory id
- feature set (subset of own/s2s)
- prepare the data

In [45]:
##> Update the traj_id of each of the data.own in the structure of f"{mmsi}.{day}.{traj_id}" <##

#> Loop over the data objects list
for ft in data_selected:
    mmsi = ft.mmsi
    new_col = ft.own["mmsi"].astype(str) + "." + ft.own["traj_id"].astype(str)
    ft.own["traj_id"] = new_col

In [46]:
own_feature_list = [d.own for d in data] # list of df

+ Store in TrajectoryCollection

In [47]:
traj_list = []
for ft in data:
    df = ft.own
    geometry = [Point(xy) for xy in zip(df['inter_lon'], df['inter_lat'])] # ! lon->lat and not lat->lon
    gdf = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)
    traj_list.append(mpd.Trajectory(gdf, traj_id='traj_id', obj_id='mmsi' ,t='t'))

In [48]:
tc = mpd.TrajectoryCollection(traj_list, traj_id_col='traj_id', t='t', crs='epsg:4326')

In [49]:
##> Check trajectories validity <##

valid_list = []
for traj in tc.trajectories:
    valid_list.append(traj.is_valid())
    # print(traj.crs)
all(valid_list)

True

+ Prepare the dataframe of features to cluster in ``concatenated_df``

In [50]:
own_df_list = []
for traj in tc.trajectories:
    # traj.df['datetime'] = traj.df.index
    own_df_list.append(traj.df)

# Concatenate the DataFrames into one DataFrame
concatenated_df = pd.concat([df.reset_index() for df in own_df_list], ignore_index=True)

# # Set the original index as a new column with the same index name
# concatenated_df['index_name'] = [df.index.name for df in df_list]

# Create a new index in the order of concatenation
concatenated_df.index = range(len(concatenated_df))

# Optionally, set a new index name (e.g., 'new_index')
concatenated_df.index.name = 'new_index'

In [51]:
concatenated_df.describe()

/home/lha/miniconda3/envs/captnAIS/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/lha/miniconda3/envs/captnAIS/lib/python3.11/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)
/home/lha/miniconda3/envs/captnAIS/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/lha/miniconda3/envs/captnAIS/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/lha/miniconda3/envs/captnAIS/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,t,epoch,status,heading,course,speed,turn,maneuver,roa,inter_speed,inter_turn,mmsi,calc_speed,direction,angular_difference,calc_acc,inter_lat,inter_lon,in_waterways,distance_shore
count,33414,3.341400e+04,33414.000000,33414.0,33414.00000,3.341400e+04,33414.0,33414.0,33414.000000,33414.000000,33414.000000,3.341400e+04,33414.000000,33414.000000,33414.000000,33414.000000,33414.000000,33414.000000,33414.000000,33414.000000
mean,2022-07-05 18:10:36.920991232,1.657045e+09,5.391273,inf,inf,inf,-inf,0.0,0.151463,5.819218,-1.378730,2.347791e+08,7.218091,162.608554,5.124914,-1.871856,54.381320,10.179579,0.448465,0.751139
min,2022-07-05 15:00:06,1.657033e+09,0.000000,0.0,0.00000,9.997559e-02,-128.0,0.0,0.000000,0.099976,-128.000000,2.095310e+08,0.000000,0.000000,0.000000,-14878.792492,54.315067,10.134866,0.000000,0.000000
25%,2022-07-05 16:20:12,1.657038e+09,0.000000,221.0,38.90625,2.400391e+00,-128.0,0.0,0.000000,2.400391,-128.000000,2.113999e+08,2.833588,57.804113,0.278507,-82.298998,54.355096,10.157755,0.000000,0.218583
50%,2022-07-05 17:39:36,1.657043e+09,0.000000,511.0,181.87500,5.800781e+00,0.0,0.0,0.000000,5.757655,0.000000,2.118481e+08,7.024570,180.000000,0.857965,0.000000,54.375301,10.178608,0.000000,0.609703
75%,2022-07-05 19:34:24,1.657050e+09,15.000000,511.0,227.46875,8.296875e+00,128.0,0.0,0.000000,8.296875,128.000000,2.460840e+08,10.367142,239.363714,2.495733,77.495793,54.413741,10.198820,1.000000,1.028483
max,2022-07-05 23:59:54,1.657066e+09,15.000000,511.0,360.00000,1.023125e+02,128.0,0.0,4.000000,102.312500,128.000000,5.111007e+08,60.874779,359.898580,180.000000,30454.401803,54.452809,10.294241,1.000000,3.636258
std,NaN,8.250257e+03,6.720893,inf,106.75000,4.132812e+00,107.5,0.0,0.387472,4.124752,106.700326,4.513044e+07,5.444598,103.720853,17.926840,717.544585,0.036838,0.025361,0.497344,0.662401


In [52]:
all_features_cols = ['t', 'mmsi', 'traj_id', 'epoch', 'geometry', 
                 'calc_speed', 'direction', 'inter_turn', 'roa', 'angular_difference', 'calc_acc', 'inter_lat', 'inter_lon', 'in_waterways', 'distance_shore']

# choose subset to cluster in
features_cols = ['t', 'mmsi', 'traj_id', 'epoch', 'geometry', 
                 'calc_speed', 'direction', 'distance_shore']

In [53]:
features_subset_cols =features_cols[5::]
X_full = concatenated_df.loc[:, features_cols]
X = concatenated_df.loc[:, features_subset_cols]

## Cluster

### HDBSCAN Clustering

In [54]:
metric = 'cityblock' # List of metrics here: https://bit.ly/45S7ESS

In [55]:
# Perform HDBSCAN clustering on the features
clusterer_manhattan = HDBSCAN(min_cluster_size=100, cluster_selection_epsilon=5, n_jobs=-1, metric=metric)
X['cluster_manhattan'] = clusterer_manhattan.fit_predict(X)

In [56]:
len(X['cluster_manhattan'].unique())

4

In [57]:
# Calculate summary statistics for each cluster
cluster_properties = X.groupby('cluster_manhattan').agg(
                        # Speed
                        min_speed=pd.NamedAgg(column='calc_speed', aggfunc='min'),
                        max_speed=pd.NamedAgg(column='calc_speed', aggfunc='max'),
                        mean_speed=pd.NamedAgg(column='calc_speed', aggfunc='mean'),
                        median_speed=pd.NamedAgg(column='calc_speed', aggfunc='median'),
                        std_speed=pd.NamedAgg(column='calc_speed', aggfunc='std'),
                        
                        # # direction
                        # min_direction=pd.NamedAgg(column='direction', aggfunc='min'),
                        # max_direction=pd.NamedAgg(column='direction', aggfunc='max'),
                        # mean_direction=pd.NamedAgg(column='direction', aggfunc='mean'),
                        # median_direction=pd.NamedAgg(column='direction', aggfunc='median'),
                        # std_direction=pd.NamedAgg(column='direction', aggfunc='std'),
                        
                        # # inter_turn
                        # min_inter_turn=pd.NamedAgg(column='inter_turn', aggfunc='min'),
                        # max_inter_turn=pd.NamedAgg(column='inter_turn', aggfunc='max'),
                        # mean_inter_turn=pd.NamedAgg(column='inter_turn', aggfunc='mean'),
                        # median_inter_turn=pd.NamedAgg(column='inter_turn', aggfunc='median'),
                        # std_inter_turn=pd.NamedAgg(column='inter_turn', aggfunc='std'),
                        
                        # # roa
                        # min_roa=pd.NamedAgg(column='roa', aggfunc='min'),
                        # max_roa=pd.NamedAgg(column='roa', aggfunc='max'),
                        # mean_roa=pd.NamedAgg(column='roa', aggfunc='mean'),
                        # median_roa=pd.NamedAgg(column='roa', aggfunc='median'),
                        # std_roa=pd.NamedAgg(column='roa', aggfunc='std'),
                        
                        # # angular_difference
                        # min_angular_difference=pd.NamedAgg(column='angular_difference', aggfunc='min'),
                        # max_angular_difference=pd.NamedAgg(column='angular_difference', aggfunc='max'),
                        # mean_angular_difference=pd.NamedAgg(column='angular_difference', aggfunc='mean'),
                        # median_angular_difference=pd.NamedAgg(column='angular_difference', aggfunc='median'),
                        # std_angular_difference=pd.NamedAgg(column='angular_difference', aggfunc='std'),
                        
                        # # calc_acc
                        # min_calc_acc=pd.NamedAgg(column='calc_acc', aggfunc='min'),
                        # max_calc_acc=pd.NamedAgg(column='calc_acc', aggfunc='max'),
                        # mean_calc_acc=pd.NamedAgg(column='calc_acc', aggfunc='mean'),
                        # median_calc_acc=pd.NamedAgg(column='calc_acc', aggfunc='median'),
                        # std_calc_acc=pd.NamedAgg(column='calc_acc', aggfunc='std'),
                        
                        # # in_waterways
                        # min_in_waterways=pd.NamedAgg(column='in_waterways', aggfunc='min'),
                        # max_in_waterways=pd.NamedAgg(column='in_waterways', aggfunc='max'),
                        # mean_in_waterways=pd.NamedAgg(column='in_waterways', aggfunc='mean'),
                        # median_in_waterways=pd.NamedAgg(column='in_waterways', aggfunc='median'),
                        # std_in_waterways=pd.NamedAgg(column='in_waterways', aggfunc='std'),
                        
                        # distance_shore
                        min_distance_shore=pd.NamedAgg(column='distance_shore', aggfunc='min'),
                        max_distance_shore=pd.NamedAgg(column='distance_shore', aggfunc='max'),
                        mean_distance_shore=pd.NamedAgg(column='distance_shore', aggfunc='mean'),
                        median_distance_shore=pd.NamedAgg(column='distance_shore', aggfunc='median'),
                        std_distance_shore=pd.NamedAgg(column='distance_shore', aggfunc='std'),
    # Add more properties as needed
).reset_index()

# Display cluster properties
print(cluster_properties)


   cluster_manhattan  min_speed  max_speed  mean_speed  median_speed  \
0                 -1   0.079528  60.874779   14.244897     13.535769   
1                  0   0.035499  20.505140    6.494036      6.532186   
2                  1   0.050441   8.386916    4.204765      6.025538   
3                  2   0.000000  18.713305    6.850090      7.221622   

   std_speed  min_distance_shore  max_distance_shore  mean_distance_shore  \
0   9.781785            0.000234            3.636258             0.965530   
1   4.326685            0.000000            3.596600             0.682714   
2   3.113539            0.004412            1.629033             0.576809   
3   4.528777            0.000311            3.277963             0.808012   

   median_distance_shore  std_distance_shore  
0               0.765529            0.810881  
1               0.547883            0.611992  
2               0.792498            0.385324  
3               0.638090            0.686384  


+ Stack the clusters back to the ``TrajectoryCollection``

In [58]:
##> Add the new clustered column to the trajectories
start_index = 0
for traj in tc.trajectories:
    stop_index = start_index + len(traj.df)
    # print(len(traj.df))
    # print(start_index, stop_index)
    traj.df['cluster_manhattan'] = X.iloc[start_index:stop_index]['cluster_manhattan'].values
    start_index = stop_index

+ Plot

In [59]:
# Create a HoloViews plot of the clusters
tc_all_manhattan_cluster_plot = tc.to_line_gdf().hvplot(geo=True,
                                                        tiles='ESRI',
                                                        title='tc',
                                                        color='cluster_manhattan',
                                                        # colorbar=True,
                                                        cmap='Viridis',
                                                        line_width=2,
                                                        alpha=0.6,
                                                        frame_width=700,
                                                        frame_height=500,
                                                        legend=True)
hvplot.save(tc_all_manhattan_cluster_plot, 
            'tc_cityblock_cluster_plot.html', 
            fmt='html')

### KMeans

In [60]:
metric = 'cityblock' # List of metrics here: https://bit.ly/45S7ESS

In [61]:
clusterer_kmeans = KMeans(n_clusters=5, random_state=0, n_init="auto")
X['cluster_kmeans'] = clusterer_kmeans.fit_predict(X)

In [62]:
print(X['cluster_kmeans'].unique())

[3 0 4 2 1]


In [63]:
# Calculate summary statistics for each cluster
cluster_properties = X.groupby('cluster_kmeans').agg(
                        # Speed
                        min_speed=pd.NamedAgg(column='calc_speed', aggfunc='min'),
                        max_speed=pd.NamedAgg(column='calc_speed', aggfunc='max'),
                        mean_speed=pd.NamedAgg(column='calc_speed', aggfunc='mean'),
                        median_speed=pd.NamedAgg(column='calc_speed', aggfunc='median'),
                        std_speed=pd.NamedAgg(column='calc_speed', aggfunc='std'),
                        
                        # distance_shore
                        min_distance_shore=pd.NamedAgg(column='distance_shore', aggfunc='min'),
                        max_distance_shore=pd.NamedAgg(column='distance_shore', aggfunc='max'),
                        mean_distance_shore=pd.NamedAgg(column='distance_shore', aggfunc='mean'),
                        median_distance_shore=pd.NamedAgg(column='distance_shore', aggfunc='median'),
                        std_distance_shore=pd.NamedAgg(column='distance_shore', aggfunc='std'),
                        
                        # direction
                        min_direction_shore=pd.NamedAgg(column='direction', aggfunc='min'),
                        max_direction_shore=pd.NamedAgg(column='direction', aggfunc='max'),
                        mean_direction_shore=pd.NamedAgg(column='direction', aggfunc='mean'),
                        median_direction_shore=pd.NamedAgg(column='direction', aggfunc='median'),
                        std_direction_shore=pd.NamedAgg(column='direction', aggfunc='std'),
    # Add more properties as needed
).reset_index()

# Display cluster properties
print(cluster_properties)


   cluster_kmeans  min_speed  max_speed  mean_speed  median_speed  std_speed  \
0               0   0.050442  54.842949    8.720161      7.785206   5.520396   
1               1   0.000000  56.272063    7.523711      7.494838   5.650112   
2               2   0.035499  22.428528    6.342586      6.766302   3.558747   
3               3   0.050442  21.498108    6.101312      4.734058   5.313123   
4               4   0.035834  60.874779    6.093738      4.779996   5.824306   

   min_distance_shore  max_distance_shore  mean_distance_shore  \
0            0.000000            3.636258             0.819485   
1            0.000311            3.316839             0.829825   
2            0.000078            2.589466             0.808610   
3            0.000201            2.476692             0.613708   
4            0.000078            3.020580             0.501994   

   median_distance_shore  std_distance_shore  min_direction_shore  \
0               0.678005            0.708200         

In [64]:
##> Add the new clustered column to the trajectories
start_index = 0
for traj in tc.trajectories:
    stop_index = start_index + len(traj.df)
    # print(len(traj.df))
    # print(start_index, stop_index)
    traj.df['cluster_kmeans'] = X.iloc[start_index:stop_index]['cluster_kmeans'].values
    start_index = stop_index

In [65]:
# Create a HoloViews plot of the clusters
tc_all_kmeans_cluster_plot = tc.to_line_gdf().hvplot(geo=True,
                                                        tiles='OSM',
                                                        title='Sailing Boats Features Cluster - KMeans',
                                                        color='cluster_kmeans',
                                                        # colorbar=True,
                                                        cmap='hsv',
                                                        line_width=2,
                                                        alpha=0.6,
                                                        frame_width=1200,
                                                        frame_height=1200,
                                                        legend=True)
hvplot.save(tc_all_kmeans_cluster_plot, 
            'tc_sailing_kmeans_cluster_plot.html', 
            fmt='html')